In [54]:
import os
import cv2
import yaml
from ultralytics import YOLO
from albumentations import Compose, HorizontalFlip, ShiftScaleRotate, Affine
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

In [55]:
# 1. Dataset YAML dosyasını oluşturma
dataset_yaml_path = "C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/dataset.yaml"
yaml_dir = os.path.dirname(dataset_yaml_path)

In [56]:
# Dizini kontrol et ve gerekirse oluştur
os.makedirs(yaml_dir, exist_ok=True)
yaml_content = {
    "path": "C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/Augmented_Data",
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "nc": 6,
    "names": ['Healthy Teeth', 'Caries', 'Impacted Teeth', 'BDC/BDR', 'Infection', 'Fractured Teeth']
}


In [57]:
# YAML dosyasını kaydet
with open(dataset_yaml_path, "w") as yaml_file:
    yaml.dump(yaml_content, yaml_file)

print(f"Dataset YAML dosyası oluşturuldu: {dataset_yaml_path}")


Dataset YAML dosyası oluşturuldu: C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/dataset.yaml


In [58]:
# CLAHE İşleme
def apply_clahe(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for img_file in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_file)
        output_path = os.path.join(output_dir, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_img = clahe.apply(img)
        cv2.imwrite(output_path, enhanced_img)

In [59]:
# Augmentasyon
def augment_image(image_path, save_path):
    image = cv2.imread(image_path)
    if image is None:
        return
    transform = Compose([
        HorizontalFlip(p=0.5),
        ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.7),
        Affine(shear=10, p=0.5),
    ])
    augmented = transform(image=image)
    cv2.imwrite(save_path, augmented["image"])

In [60]:
# Modeli Eğitme
def train_model():
    model = YOLO('yolo11n.pt')  # Daha hızlı eğitim için YOLOv8 küçük modeli kullanıldı
    model.train(
        data=dataset_yaml_path,
        epochs=100,
        batch=16,
        imgsz=640,
        name='dental_detection',
        project='C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/runs'
    )


In [61]:
# Tahmin
def predict_images():
    model = YOLO('runs/train/dental_detection/weights/best.pt')
    results = model.predict(
        source='C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/Augmented_Data/test/images',
        conf=0.3,
        save=True  # Tahmin edilen görselleri ve sonuçları kaydeder
    )
    return results

In [62]:
# Yeni Görüntü Üzerinde Tahmin Yapma
def predict_new_image(image_path):
    model = YOLO('runs/train/dental_detection/weights/best.pt')
    results = model.predict(
        source=image_path,
        conf=0.3,
        save=True  # Tahmin sonuçları otomatik olarak exp dizininde saklanır
    )
 # Tahmin edilen görüntüyü görselleştir
    exp_dir = "runs/predict"
    result_image_path = f"{exp_dir}/exp/{os.path.basename(image_path)}"
    image = cv2.imread(result_image_path)
    if image is not None:
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()
    else:
        print("Tahmin edilen görüntü bulunamadı.")

In [63]:
# Ana Fonksiyon
def main():
    try:
        input_dir = "C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/Augmented_Data/train/images"
        clahe_dir = "C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/Augmented_Data/train/images_clahe"
        augmented_dir = "C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/Augmented_Data/train/images_augmented"

        print("CLAHE uygulanıyor...")
        apply_clahe(input_dir, clahe_dir)

        print("Augmentasyon uygulanıyor...")
        os.makedirs(augmented_dir, exist_ok=True)
        for img_file in os.listdir(clahe_dir):
            augment_image(os.path.join(clahe_dir, img_file), os.path.join(augmented_dir, img_file))

        print("Eğitim başlatılıyor...")
        train_model()

        print("Tahmin başlatılıyor...")
        results = predict_images()
        print("Tahmin tamamlandı.")

    except Exception as e:
        print(f"Hata oluştu: {e}")

if __name__ == "__main__":
    main()

CLAHE uygulanıyor...
Augmentasyon uygulanıyor...
Eğitim başlatılıyor...



00%|██████████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [00:09<00:00, 621kB/s]

Ultralytics 8.3.48  Python-3.12.6 torch-2.5.1+cpu CPU (AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:/Users/Kullanc/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=C:/Users/Kullanc/Desktop/dentalyze_jupyter2/runs, name=dental_detection3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=

train: Scanning C:\Users\Kullanıcı\Desktop\dentalyze_jupyter2\Dental OPG XRAY Dataset\Augmented_Data\train\labels.cache

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<0

                   all         23        115       0.62      0.699      0.659      0.412



100 epochs completed in 11.727 hours.
Optimizer stripped from C:\Users\Kullanc\Desktop\dentalyze_jupyter2\runs\dental_detection3\weights\last.pt, 5.5MB
Optimizer stripped from C:\Users\Kullanc\Desktop\dentalyze_jupyter2\runs\dental_detection3\weights\best.pt, 5.5MB

Validating C:\Users\Kullanc\Desktop\dentalyze_jupyter2\runs\dental_detection3\weights\best.pt...
Ultralytics 8.3.48  Python-3.12.6 torch-2.5.1+cpu CPU (AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx)
YOLO11n summary (fused): 238 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs



                Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<0

                   all         23        115       0.71      0.666      0.718      0.464
         Healthy Teeth          8         14       0.54      0.755      0.609      0.421
                Caries          3          4      0.462        0.5      0.563      0.316
        Impacted Teeth          7         18      0.859      0.889      0.968      0.594
               BDC/BDR          3          3          1      0.642       0.83      0.565
             Infection          6          9      0.941      0.778      0.886      0.571
       Fractured Teeth         22         67      0.458      0.433      0.452       0.32
Speed: 5.4ms preprocess, 125.9ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to C:\Users\Kullanc\Desktop\dentalyze_jupyter2\runs\dental_detection3
Tahmin başlatılıyor...
Hata oluştu: [Errno 2] No such file or directory: 'runs\\train\\dental_detection\\weights\\best.pt'


In [107]:
import os
import random
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Modeli yükle
model_path = "C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/runs/dental_detection3/weights/best.pt"
model = YOLO(model_path)

# Rastgele bir görüntü seçme
def get_random_image_from_folder(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    if not image_files:
        raise FileNotFoundError(f"Klasörde görüntü bulunamadı: {folder_path}")
    random_image = random.choice(image_files)
    return os.path.join(folder_path, random_image)

# Orijinal bounding box çizme
def draw_bounding_boxes(image, label_path):
    with open(label_path, "r") as file:
        labels = file.readlines()

    for label in labels:
        parts = label.strip().split()
        class_id, x_center, y_center, width, height = map(float, parts)

        # YOLO formatından (x_center, y_center, width, height) piksel koordinatlarına dönüştür
        img_h, img_w = image.shape[:2]
        x1 = int((x_center - width / 2) * img_w)
        y1 = int((y_center - height / 2) * img_h)
        x2 = int((x_center + width / 2) * img_w)
        y2 = int((y_center + height / 2) * img_h)

        # Sınıf bilgisi eklenerek bounding box çiz
        color = (0, 255, 0)  # Yeşil renk (orijinal kutu)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f"Class {int(class_id)}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return image

# Tahmin işlemi ve karşılaştırma
def predict_and_compare(folder_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)  # Çıktı klasörü oluştur
    image_path = get_random_image_from_folder(folder_path)  # Rastgele bir görüntü seç
    print(f"Seçilen rastgele görüntü: {image_path}")

    # Etiket dosyasını bul
    label_path = image_path.replace(".jpg", ".txt").replace(".png", ".txt").replace(".jpeg", ".txt")
    if not os.path.exists(label_path):
        raise FileNotFoundError(f"Label dosyası bulunamadı: {label_path}")

    # Orijinal görüntüyü bounding box'larla çiz
    original_image = cv2.imread(image_path)
    original_image_with_boxes = draw_bounding_boxes(original_image.copy(), label_path)

    # Tahmin işlemi
    results = model.predict(
        source=image_path,
        conf=0.3,
        save=True,
        project="C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/runs/predict",
        name="predict"
    )

    # Tahmin edilen görüntünün kaydedildiği dizin
    save_dir = results[0].save_dir if hasattr(results[0], "save_dir") else "C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/runs/predict/predict"
    print(f"Tahmin edilen görüntülerin kaydedildiği dizin: {save_dir}")

    # Tahmin edilen görüntünün tam yolu
    result_image_path = os.path.join(save_dir, os.path.basename(image_path))
    predicted_image = cv2.imread(result_image_path)

    # Görselleştirme ve kaydetme
    if original_image_with_boxes is not None and predicted_image is not None:
        plt.figure(figsize=(18, 8))

        # Orijinal bounding box'lı görüntü
        plt.subplot(1, 2, 1)
        plt.title("Orijinal Bounding Box'lı Görüntü")
        plt.imshow(cv2.cvtColor(original_image_with_boxes, cv2.COLOR_BGR2RGB))
        plt.axis("off")

        # Tahmin edilen bounding box'lı görüntü
        plt.subplot(1, 2, 2)
        plt.title("Tahmin Edilen Bounding Box'lı Görüntü")
        plt.imshow(cv2.cvtColor(predicted_image, cv2.COLOR_BGR2RGB))
        plt.axis("off")

        # Görüntüyü kaydet
        output_path = os.path.join(output_dir, "comparison_with_boxes.png")
        plt.savefig(output_path)  # Karşılaştırma görselini kaydet
        print(f"Karşılaştırma görseli kaydedildi: {output_path}")

        # Görselleştir
        plt.show()

    else:
        print("Görüntüler yüklenemedi veya tahmin edilen görüntü bulunamadı.")

# Kullanım
folder_path = "C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/Original_Data"
output_dir = "C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/runs/visualization"
predict_and_compare(folder_path, output_dir)


Seçilen rastgele görüntü: C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/Dental OPG XRAY Dataset/Original_Data\207.jpg

image 1/1 C:\Users\Kullanc\Desktop\dentalyze_jupyter2\Dental OPG XRAY Dataset\Original_Data\207.jpg: 320x640 1 Infection, 5 Fractured Teeths, 226.2ms
Speed: 8.2ms preprocess, 226.2ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
Results saved to C:\Users\Kullanc\Desktop\dentalyze_jupyter2\runs\predict\predict8
Tahmin edilen görüntülerin kaydedildiği dizin: C:\Users\Kullanıcı\Desktop\dentalyze_jupyter2\runs\predict\predict8
Karşılaştırma görseli kaydedildi: C:/Users/Kullanıcı/Desktop/dentalyze_jupyter2/runs/visualization\comparison_with_boxes.png


<Figure size 1800x800 with 2 Axes>